In [1]:
import pandas as pd 
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression

In [2]:
#Parameters
file = "../tables/2021AP.xlsx"
next_week = 7
#Read in df
df = pd.read_excel(file)
df

Team  Week  Rank  W  L  Winning Percentage  Opp. Rank  \
0            Alabama     1     1  0  0                   1         14   
1           Oklahoma     1     2  0  0                   1         26   
2            Clemson     1     3  0  0                   1          5   
3         Ohio State     1     4  0  0                   1         26   
4            Georgia     1     5  0  0                   1          3   
..               ...   ...   ... .. ..                 ...        ...   
171         NC State     7    22  4  1                 0.8        NaN   
172              SMU     7    23  6  0                   1        NaN   
173  San Diego State     7    24  5  0                   1        NaN   
174            Texas     7    25  4  2            0.666667        NaN   
175             Team  Week  Rank  W  L  Winning Percentage  Opp. Rank   

     Opp. P5  Home  Result  Points Scored  Points Against  Margin  \
0          1     0       W             44              13      31   
1          0     1       W             40              35       5   
2          1     0       L              3              10      -7   
3          1     0       W             45              31      14   
4          1     0       W             10               3       7   
..       ...   ...     ...            ...             ...     ...   
171      NaN   NaN     NaN            NaN             NaN     NaN   
172      NaN   NaN     NaN            NaN             NaN     NaN   
173      NaN   NaN     NaN            NaN             NaN     NaN   
174      NaN   NaN     NaN            NaN             NaN     NaN   
175  Opp. P5  Home  Result  Points Scored  Points Against  Margin   

     Next Week Rank  Movement  
0                 1         0  
1                 4        -2  
2                 6        -3  
3                 3         1  
4                 2         3  
..              ...       ...  
171             NaN       NaN  
172             NaN       NaN  
173             NaN       NaN  
174             NaN       NaN  
175  Next Week Rank  Movement  

[176 rows x 15 columns]

In [3]:
#Drop footer
df.drop(index = (df.index.stop - 1), inplace = True)

#Drop overflow
for index, row in df.iterrows():
    if row['Week'] >= next_week:
        df.drop(index = index, inplace = True)

In [4]:
#Encode Result column
for index, row in df.iterrows():
    if row['Result'] == 'W':
        df.at[index, 'Result'] = 1
    else: 
        df.at[index, 'Result'] = 0

In [5]:
#Get next week teams list
next_teams = df['Team'].loc[df['Week'] == next_week - 1]
next_rank_act = df['Next Week Rank'].loc[df['Week'] == next_week - 1]

#Get byes
byes_index = []
for index, row in df.iterrows():
    if row['Week'] == next_week - 1 and not(row['Opp. Rank'] >= 0):
        byes_index.append(index % 25)
        
byes_index

[7, 11, 22]

In [6]:
#Drop null columns (teams with byes)
df.dropna(inplace = True)

In [7]:
#Drop irrelevant cols
df = df.drop(columns = ['Movement', 'Team'])

In [8]:
#Convert cols to num
for name, values in df.iteritems():
    df[name] = pd.to_numeric(values)

In [9]:
#Set training data to Week < next week
train_df = df.loc[df['Week'] < next_week - 1]
X_train = train_df.drop(columns = ['Next Week Rank'])
y_train = train_df['Next Week Rank']

In [10]:
#Set testing data to Week = next week
test_df = df.loc[df['Week'] == next_week - 1]
X_test = test_df.drop(columns = ['Next Week Rank'])
y_test = test_df['Next Week Rank']

In [11]:
#Multiple Linear Regression
lin_model = LinearRegression().fit(X_train, y_train)
lin_model_output = lin_model.predict(X_test)
lin_model.score(X_test, y_test)

0.8929108833254176

In [12]:
#Add back teams with byes to output
next_rank_pred = lin_model_output.tolist()
for item in byes_index:
    next_rank_pred.insert(item, 'bye')

In [13]:
model_output = lin_model_output.tolist()
model_output

for item in byes_index:
    model_output.insert(item + 1, item + 1)
    
model_output.sort()    

sorted_output = model_output

In [14]:
df2 = pd.DataFrame()
df2['Predicted Rank'] = sorted_output
df2['Predicted Ordinal Rank'] = range(1, 26)

In [15]:
results = pd.DataFrame()
results['Team'] = next_teams
results['Predicted Rank'] = next_rank_pred
results['Actual Rank'] = next_rank_act

results.reset_index(inplace = True, drop = True)

#Predict that teams with byes don't change rank
for index, row in results.iterrows():
    if row['Predicted Rank'] == 'bye':
        results.at[index, 'Predicted Rank'] = index + 1

results['Predicted Rank'] = pd.to_numeric(results['Predicted Rank'], errors = 'coerce')
results['Actual Rank'] = pd.to_numeric(results['Actual Rank'])
results['Previous Rank'] = range(1, 26)

merged_results = results.merge(df2, how = 'left', left_on = 'Predicted Rank', right_on = 'Predicted Rank')

In [16]:
merged_results = merged_results[['Team', 'Previous Rank', 'Predicted Rank', 'Predicted Ordinal Rank', 'Actual Rank']]

In [17]:
merged_results

Team  Previous Rank  Predicted Rank  Predicted Ordinal Rank  \
0            Alabama              1       11.313245                      10   
1            Georgia              2       -0.027126                       2   
2               Iowa              3       -1.893372                       1   
3         Penn State              4        8.192643                       8   
4         Cincinnati              5        6.020401                       4   
5           Oklahoma              6        4.020185                       3   
6         Ohio State              7        6.798468                       5   
7             Oregon              8        8.000000                       7   
8           Michigan              9        7.582949                       6   
9                BYU             10       18.845420                      17   
10    Michigan State             11        8.534516                       9   
11    Oklahoma State             12       12.000000                      13   
12          Arkansas             13       20.125007                      21   
13        Notre Dame             14       13.573748                      14   
14  Coastal Carolina             15       11.679675                      11   
15          Kentucky             16       11.925687                      12   
16          Ole Miss             17       14.767050                      16   
17            Auburn             18       21.878974                      23   
18       Wake Forest             19       14.752606                      15   
19           Florida             20       18.892233                      18   
20             Texas             21       23.535600                      25   
21     Arizona State             22       19.062301                      19   
22          NC State             23       23.000000                      24   
23               SMU             24       19.082581                      20   
24   San Diego State             25       21.349496                      22   

    Actual Rank  
0             5  
1             1  
2             2  
3             7  
4             3  
5             4  
6             6  
7             9  
8             8  
9            19  
10           10  
11           12  
12           17  
13           14  
14           15  
15           11  
16           13  
17           26  
18           16  
19           20  
20           25  
21           18  
22           22  
23           23  
24           24

In [18]:
merged_results.to_csv(f'week{next_week}_predictions.csv', header = True, index = False)